## Example

This example is from Verisig's paper, Fig. 2.

In [14]:
using Revise, NeuralNetworkAnalysis

using NeuralVerification: Layer, Id
const NNA = NeuralNetworkAnalysis
const RA = ReachabilityAnalysis

ReachabilityAnalysis

In [12]:
nnet = Network([Layer([0.3 0.2; 0.1 0.5], [0.1, 0.2], NNA.Sigmoid()), Layer(hcat(3, 5.), [0.], Id())])
dump(nnet)

Network
  layers: Array{Layer}((2,))
    1: Layer{NeuralNetworkAnalysis.Sigmoid,Float64}
      weights: Array{Float64}((2, 2)) [0.3 0.2; 0.1 0.5]
      bias: Array{Float64}((2,)) [0.1, 0.2]
      activation: NeuralNetworkAnalysis.Sigmoid NeuralNetworkAnalysis.Sigmoid()
    2: Layer{Id,Float64}
      weights: Array{Float64}((1, 2)) [3.0 5.0]
      bias: Array{Float64}((1,)) [0.0]
      activation: Id Id()


In [13]:
xᴾ = Hyperrectangle(low=[2, 1.], high=[3, 2.])

Hyperrectangle{Float64,Array{Float64,1},Array{Float64,1}}([2.5, 1.5], [0.5, 0.5])

In [10]:
@time forward(nnet, xᴾ)

  0.419473 seconds (3.65 M allocations: 281.212 MiB, 14.40% gc time)


CartesianProductArray{Float64,Interval{Float64,IntervalArithmetic.Interval{Float64}}}(Interval{Float64,IntervalArithmetic.Interval{Float64}}[Interval{Float64,IntervalArithmetic.Interval{Float64}}([5.6848, 6.52503])])

---

## Using zonotopes

In [26]:
@taylorize function sigmoid2!(dx, x, p, t)
    xᴶ₁, xᴶ₂, xᴾ₁, xᴾ₂ = x
    dx[1] = zero(xᴶ₁)
    dx[2] = zero(xᴶ₂)
    
    dx[3] = xᴶ₁ *(xᴾ₁ - xᴾ₁^2)
    dx[4] = xᴶ₂ *(xᴾ₂ - xᴾ₂^2)
end

# Method: Zonotope
function forward_zono(nnet::Network, X0::LazySet;
                      alg=TMJets(abs_tol=1e-14, orderQ=2, orderT=6))

    # initial states
    xᴾ₀ = RA._convert_or_overapproximate(X0, Zonotope)

    for layer in nnet.layers  # loop over layers
        W = layer.weights
        m, n = size(W)
        b = layer.bias
        act = layer.activation

        xᴶ′ = affine_map(W, xᴾ₀, b)
        xᴾ′ = CartesianProductArray(fill(Interval(0.5, 0.5), m))

        if act == Id()
            xᴾ₀ = copy(xᴶ′)
            continue
        end
        @assert act == NNA.Sigmoid()

        ivp = @ivp(x' = sigmoid2!(x), dim=4, x(0) ∈ xᴶ′ × xᴾ′)
        sol = RA.solve(ivp, tspan=(0., 1.), alg=alg)
        Zend = overapproximate(sol[end], Zonotope)
        xᴾ₀ = project(Zend, m+1:2m) |> set
    end
    return xᴾ₀
end

forward_zono (generic function with 1 method)

In [29]:
xᴾ = Hyperrectangle(low=[2, 1.], high=[3, 2.])
@time out = forward_zono(nnet, xᴾ)

  0.299355 seconds (2.29 M allocations: 184.792 MiB, 13.62% gc time)


Zonotope{Float64,Array{Float64,1},Array{Float64,2}}([6.120451474048576], [0.1369676214119743 0.2668022483155346 0.00029308398132414626 0.0005023896202388792])

In [31]:
y = overapproximate(out, Interval)

Interval{Float64,IntervalArithmetic.Interval{Float64}}([5.71588, 6.52502])

Observations:

- If TMJets received a zonotope the precision sohuld increase, in general.
- The approach with zonotopes gives $[5.71588, 6.52502]$ which is tighter than $[5.6848, 6.52503]$ obtained with a cartesian decompositon of intervals.

---

Questions:

- How do the methods scale?
- How to define `sigmoid2!` programmatically, ie. for varying number of dimensions.